# Fetching and filtering data from the database
    
When using the dices-client you will be able to manipulate data by using predefined methods to parse and manipulate data.
    
You will be able to use DataGroups more effectively with reading this notebook. This document will go over the datatypes that are included in the client for you to better understand how to efficiently parse the data given from the database.

## Preliminaries

First, we have to create a connection to the DICES API:

In [1]:
import dicesapi
from dicesapi import DicesAPI
from dicesapi.jupyter import NotebookPBar

api = DicesAPI(
    progress_class = NotebookPBar,
    logfile='dices.log',
)

## Datagroups
    
All methods that return a group of Datatypes will return a Datagroup.

e.g. `.getSpeeches()` would return a DataGroup:

In [2]:
speeches = api.getSpeeches(spkr_name='Achilles', progress=True)

In [3]:
isinstance(speeches, dicesapi._DataGroup)

True

There are Datagroups for each data class that the API provides (EG: CharacterGroup, SpeechGroup, CharacterInstanceGroup etc...) and each method that fetches data yields its respective Datagroup child (`.getSpeeches()` => SpeechGroup)

In [4]:
type(speeches)

dicesapi._SpeechGroup

These DataGroups have plenty of built-in functionality to them that allows them to be manipulated.
    
## List-like behaviour
 
In essence, DataGroups can be treated similarly to lists, where you can fetch data from them in similar ways to lists.

### Fetching an element of the datagroup

In [5]:
print('first speech: ' + str(speeches[0]))
print('sixth speech: ' + str(speeches[5]))

first speech: <Speech 4: Iliad 1.59-1.67>
sixth speech: <Speech 15: Iliad 1.216-1.218>


### Iterating over the DataGroup

In [6]:
for i, s in enumerate(speeches[:5]):
    print(f'item {i}: {s}')

item 0: <Speech 4: Iliad 1.59-1.67>
item 1: <Speech 6: Iliad 1.85-1.91>
item 2: <Speech 9: Iliad 1.122-1.129>
item 3: <Speech 11: Iliad 1.149-1.171>
item 4: <Speech 13: Iliad 1.202-1.205>


### Get number of elements in the DataGroup

In [7]:
print(len(speeches))

111


## Operators

### Addition

Addition allows you to concatenate two DataGroups together. This method will remove duplicate values. Addition creates a brand new DataGroup consisting of elements of both datagroups.

In [8]:
ach_spkr = api.getSpeeches(spkr_name='Achilles', work_title='Iliad')
ach_addr = api.getSpeeches(addr_name='Achilles', work_title='Iliad')

print(f'Achilles is speaker in {len(ach_spkr)} speeches in the Iliad.')
print(f'Achilles is addressee in {len(ach_addr)} speeches in the Iliad.')

all_ach = ach_spkr + ach_addr
print(f'Achilles is involved in {len(all_ach)} speeches in the Iliad (in any role).')

Achilles is speaker in 88 speeches in the Iliad.
Achilles is addressee in 51 speeches in the Iliad.
Achilles is involved in 134 speeches in the Iliad (in any role).


**Notes**

- If you want to keep duplicates use `.extend(otherDataGroup, duplicates=True)`

- If you want to add group2 directly to group1, use: `group1 += group2`

### Subtraction

Subtraction creates a new DataGroup that takes the elements from DataGroup1 and excludes any that are a part of DataGroup2.

In [9]:
# speeches where Ach is not speaking to himself

ach_to_others = ach_spkr - ach_addr
print(len(ach_to_others))

83


**Notes**

- If you want to modify group1 directly instead of making a copy, use: `group1 -= group2`

## Methods

### General Purpose

#### `extend(DataGroup, duplicates=False)`

Extend acts similarly to `+=`, however it allows the user to specify if duplicate values should be left in the list. In most cases `+` should be used but if duplicates are needed extend must be used.

#### `intersect(DataGroup)`

Intersect takes the elements from two DataGroups and returns a new DataGroup that only contains elements that were shared in both DataGroups

In [10]:
# only speeches where Achilles talks to himself
ach_self = ach_spkr.intersect(ach_addr)

print(len(ach_self))

5


### Filters

Filters are methods that allows users to take DataGroups and find elements that have specific values attached to them. Every DataGroup child has their own specific filters, however there are some that are shared between all DataGroups.
    
To learn more about filters check the "Using and Understanding Filters" notebook

#### `filterAttribute(attribute, value)`

filterAttribute allows users to search through all the elements of a DataGroup and filters them based on an inputted attribute and matching value. This method only checks for literal values (This includes Strings, integers and floats)

In [11]:
# only speeches that initiate a conversation
filtered = ach_spkr.filterAttribute('part', 1)
print(len(filtered))

47


#### `filterList(attribute, list)`

filterList allows users to search through all the elements of a DataGroup and filters them based on an inputted attribute, and then checks if the value at this attribute is contained withing a supplied list. This allows users to include multiple values in their filter

In [12]:
# only speeches that represent an even-numbered turn
filtered = ach_spkr.filterList('part', [2, 4, 6, 8, 10, 12])
print(len(filtered))

25


#### `advancedFilter(func)`

advancedFilter allows users to provide their own filter function to filter all of the things that are in the given data, these filters can be anything as long as they return a boolean values and have a single argument which will be an element of the DataGroup. For plenty of examples of the usage of this method, please check the "Using and Understanding Filters" notebook

In [13]:
# a better way to check even numbered turns,
# especially if you're going to use it more than once

def filterFunction(speech):
    '''check whether speech is even-numbered turn'''
    return speech.part % 2 == 0

filtered = ach_spkr.advancedFilter(filterFunction)
print(len(filtered))

25


In [14]:
# for one-offs, use lambda

filtered = ach_spkr.advancedFilter(lambda s: s.part % 2 == 0)
print(len(filtered))

25
